# Lithops Pi number Montecarlo approximation example
In this notebook we will calculate an approximation to the number Pi by applying the Montecarlo algorithm with Lithops. Functions will process a fixed amount of random samples each that will then be averaged to provide the approximation.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lithops.multiprocessing import Pool, Queue

%matplotlib notebook

In [3]:
N = 100
iter_per_func = 1000
chosen_func_mod = N // 2
num_points_to_send = 20

### Monitoring
Some functions will be chosen to send some of their already classified points over a queue that the main process will be consuming from to provide a live image of the first results.

In [4]:
def pi_montecarlo(n, q):  
    l = list()
    value = 0
    chosen = n % chosen_func_mod == 0
    
    for i in range(iter_per_func):
        # Generate random point between 0 and 1
        x = np.random.rand() 
        y = np.random.rand()
        z = np.sqrt(x * x + y * y)
        if z <= 1:
            # Point is inside circle
            value += 1

        if chosen:
            l.append((x, y, z))
            if i == num_points_to_send:
                # Send generated points
                q.put(l)
                chosen = False
            
    est_pi = value * 4.0 / iter_per_func
    return est_pi

### Configure access to your Cloud Storage and Cloud Functions

Configure access details to your AWS or GCP Cloud Functions.  'storage_bucket'  should point to some pre-existing bucket. This bucket will be used by Lithops to store intermediate results. All results will be stored in the folder `lithops.jobs`. For additional configuration parameters see [configuration section](https://github.com/lithops-cloud/lithops)

For GCP your `.lithops_config` should be similar to: 
    
    lithops:
        storage: redis
        backend: gcp_functions
        bucket: lithops-pipelines
    
    gcp:
        credentials_path : <PATH_TO_JSON_KEYS>
        region : <GCP_REGION>
    
    gcp_functions:
        region : <GCP_REGION>
    
    redis:
        host : <REDIS_HOST_IP>
        port : <REDIS_HOST_PORT>
        password: <REDIS_PASSWORD>

For AWS your `.lithops_config` should be similar to: 
    
    lithops:
        storage: redis
        backend: aws_lambda
    
    aws:
        access_key_id : <AWS_ACCESS_KEY_ID>
        secret_access_key : <AWS_SECRET_ACCESS_KEY> 
    
    aws_lambda:
        execution_role: <AWS_ROLE_ARN>
        region_name: <REGION>
        
    redis:
        host : <REDIS_HOST_IP>
        port : <REDIS_HOST_PORT>
        password: <REDIS_PASSWORD>

### Execution

In [ ]:
pool = Pool()
queue = Queue()
results = pool.starmap_async(func=pi_montecarlo, iterable=[(i, queue) for i in range(N)])

### Plot results on the go

In [ ]:
fig, ax = plt.subplots(figsize=(7.5,7.5))
plt.ion()
fig.canvas.draw()
plt.show()

n_messages = 0
num_chosen_func = N // chosen_func_mod

while n_messages < num_chosen_func:
    l = queue.get()  
    for x, y, z in l:
        color = 'red' if z <= 1 else 'blue'
        ax.scatter(x, y, c=color, s=2)
    n_messages += 1
    plt.title("Displaying {:,d} of {:,d} generated points".format(n_messages*num_points_to_send, N*iter_per_func))
    fig.canvas.draw()

plt.close()

## Pi estimation
Finally, we try to calculate the number Pi by averaging the results of each function.  
The more iterations we perform, the more acurate the approximation becomes.

In [ ]:
est_pi = np.mean(results.get())
print('Estimated pi: {}'.format(est_pi))
print('Num iterations: {:,d}'.format(N*iter_per_func))